In [56]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [57]:
df = pd.read_csv('IMDB.csv')
# df = df.sample(500)
# df.to_csv('data.csv', index=False)
# df.head()
df

,review,sentiment
0,Film version of Sandra Bernhard's one-woman of...,negative
1,I switched this on (from cable) on a whim and ...,positive
2,The `plot' of this film contains a few holes y...,negative
3,"Some amusing humor, some that falls flat, some...",negative
4,What can you say about this movie? It was not ...,negative
...,...,...
995,"Not exactly a new story line, but this romanti...",negative
996,I first saw this movie as a younger child. My ...,positive
997,Some people have stated that as of the 11th se...,positive
998,Nothing but the director's juvenile fantasy co...,negative


In [58]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [43]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ankit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [59]:
df = normalize_text(df)
df.head()

,review,sentiment
0,film version sandra bernhard s one woman off b...,negative
1,switched from cable whim treated quite surpris...,positive
2,plot film contains hole could drive massive tr...,negative
3,amusing humor fall flat decent acting quite at...,negative
4,say movie terrible good two day earlier watche...,negative


In [60]:
df['sentiment'].value_counts()

sentiment
negative    517
positive    483
Name: count, dtype: int64

In [61]:
df.shape

(1000, 2)

In [62]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [63]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
0,film version sandra bernhard s one woman off b...,0
1,switched from cable whim treated quite surpris...,1
2,plot film contains hole could drive massive tr...,0
3,amusing humor fall flat decent acting quite at...,0
4,say movie terrible good two day earlier watche...,0


In [64]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [65]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [52]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/ankit693/Recommender-system.mlflow')
dagshub.init(repo_owner='ankit693', repo_name='Recommender-system', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-10-23 22:04:45,655 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/ankit693/Recommender-system "HTTP/1.1 200 OK"


Initialized MLflow to track repo "ankit693/Recommender-system"

2025-10-23 22:04:45,673 - INFO - Initialized MLflow to track repo "ankit693/Recommender-system"


Repository ankit693/Recommender-system initialized!

2025-10-23 22:04:45,679 - INFO - Repository ankit693/Recommender-system initialized!


<Experiment: artifact_location='mlflow-artifacts:/495b19f2964a45088f39026a9170c71e', creation_time=1761236748084, experiment_id='0', last_update_time=1761236748084, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
import mlflow
import logging
import os
import time
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# --------------------------------------------------------------------
# CONFIGURE LOGGING
# --------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("🚀 Starting MLflow run...")

# --------------------------------------------------------------------
# SETUP: If you're using DagsHub or a local MLflow instance
# --------------------------------------------------------------------
# Uncomment and edit this if needed (optional)
mlflow.set_tracking_uri("https://dagshub.com/ankit693/Recommender-system.mlflow")

# You can also name your experiment
mlflow.set_experiment("Baseline_Logistic_Regression")

# --------------------------------------------------------------------
# TRAINING AND LOGGING PIPELINE
# --------------------------------------------------------------------
with mlflow.start_run():
    start_time = time.time()

    try:
        # --------------------------------------------------------------
        # Example data placeholders (REPLACE with your real data)
        # --------------------------------------------------------------
        # X_train, X_test, y_train, y_test = <your_preprocessed_data>

        # --------------------------------------------------------------
        # Log preprocessing parameters
        # --------------------------------------------------------------
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 120)
        mlflow.log_param("test_size", 0.3)

        # --------------------------------------------------------------
        # Initialize and train the model
        # --------------------------------------------------------------
        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("✅ Model training complete.")

        # --------------------------------------------------------------
        # Log model parameters
        # --------------------------------------------------------------
        mlflow.log_param("model_type", "Logistic Regression")

        # --------------------------------------------------------------
        # Predictions and Evaluation
        # --------------------------------------------------------------
        logging.info("Making predictions and calculating metrics...")
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # --------------------------------------------------------------
        # Log metrics
        # --------------------------------------------------------------
        mlflow.log_metrics({
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1
        })

        # --------------------------------------------------------------
        # Try to log the model (supports both local MLflow and DagsHub)
        # --------------------------------------------------------------
        logging.info("Saving and logging the model...")
        try:
            mlflow.sklearn.log_model(model, name="model")  # ✅ new syntax (no deprecation warning)
            logging.info("✅ Model successfully logged with MLflow.")
        except Exception as log_err:
            logging.warning(f"⚠️ MLflow model logging failed ({log_err}). Logging as artifact instead.")
            model_path = "model.pkl"
            joblib.dump(model, model_path)
            mlflow.log_artifact(model_path)
            logging.info("✅ Model logged as artifact instead.")

        # --------------------------------------------------------------
        # Log execution time
        # --------------------------------------------------------------
        end_time = time.time()
        elapsed = end_time - start_time
        mlflow.log_metric("execution_time_sec", elapsed)
        logging.info(f"⏱️ Training and logging completed in {elapsed:.2f} seconds.")

        # --------------------------------------------------------------
        # Print summary results
        # --------------------------------------------------------------
        logging.info(f"📊 Accuracy:  {accuracy:.4f}")
        logging.info(f"📊 Precision: {precision:.4f}")
        logging.info(f"📊 Recall:    {recall:.4f}")
        logging.info(f"📊 F1 Score:  {f1:.4f}")

    except Exception as e:
        logging.error(f"❌ An error occurred: {e}", exc_info=True)


2025-10-23 22:15:54,499 - INFO - 🚀 Starting MLflow run...
2025-10-23 22:15:56,504 - INFO - Logging preprocessing parameters...
2025-10-23 22:15:57,555 - INFO - Initializing Logistic Regression model...
2025-10-23 22:15:57,555 - INFO - Fitting the model...
2025-10-23 22:15:57,620 - INFO - ✅ Model training complete.
2025-10-23 22:15:57,950 - INFO - Making predictions and calculating metrics...
2025-10-23 22:15:58,324 - INFO - Saving and logging the model...
2025-10-23 22:15:58,988 - WARNING - ⚠️ MLflow model logging failed (INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}). Logging as artifact instead.
2025-10-23 22:16:00,288 - INFO - ✅ Model logged as artifact instead.
2025-10-23 22:16:00,620 - INFO - ⏱️ Training and logging completed in 3.80 seconds.
2025-10-23 22:16:00,629 - INFO - 📊 Accuracy:  0.6900
2025-10-23 22:16:00,631 - INFO - 📊 Precision: 0.7206
2025-10-23 22:16:00,632 - INFO - 📊 Recall:    0.6405
2025-10-23 22:16:00,636 - INFO - 

🏃 View run intrigued-bird-621 at: https://dagshub.com/ankit693/Recommender-system.mlflow/#/experiments/1/runs/d31d9242cbbe49e0a0ed57433740c9ce
🧪 View experiment at: https://dagshub.com/ankit693/Recommender-system.mlflow/#/experiments/1
